## Scale Up vs Scale Out
* Scale Up : 한 대의 컴퓨터 성능을 최적화
* Scale Out : 여러대의 컴퓨터를 한 대처럼 사용
    로드 밸런싱을 구현해서 병목현상을 줄여야 한다.

### 스레드 생성

In [1]:
from threading import *

class Delivery:
    def run(self):
        print('delivering')
        
work1 = Delivery()
print(work1.run)

class Delivery(Thread):
    def run(self):
        print('delivering')
        
work2 = Delivery()
print(work2.run)

<bound method Delivery.run of <__main__.Delivery object at 0x7f9248aa6c10>>
<bound method Delivery.run of <Delivery(Thread-4, initial)>>


할당된 쓰레드가 다르다.

In [3]:
# 쓰레드 생성하는 다른 방법
from threading import *
from time import sleep

Stopped = False

def worker(work, sleep_sec):
    while not Stopped:
        print('do', work)
        sleep(sleep_sec)
    print('retired...')

t = Thread(target=worker, args=('Overwork', 3))
t.start()

do Overwork
do Overwork
do Overwork
do Overwork
do Overwork
do Overwork
do Overwork
do Overwork
do Overwork
do Overwork
do Overwork


In [4]:
Stopped = True    # 일꾼 일 그만하라고 세팅해 줍시다. 
t.join()          # 일꾼 스레드가 종료할때까지 기다립니다. 
print('worker is gone.')

retired...
worker is gone.


### 프로세스 생성

In [7]:
import multiprocessing as mp

def delivery():
    print('delivering')

p=mp.Process(target=delivery, args=())
p = mp.Process(target=delivery, args=())
p.start() # 프로세스 시작
#p.join() # 실제 종료까지 기다림 (필요시에만 사용)


delivering


In [8]:
p.terminate() # 프로세스 종료

### 스레드/프로세스 풀 사용하기
##### 풀 만드는 방법
1. Queue를 사용해서 직접 생성
2. concurrent.futures 라이브러리의 ThreadPoolExecutor, ProcessPoolExecutor클래스를 이용

In [9]:
# concurrent.futures 사용
from concurrent.futures import ThreadPoolExecutor

class Delivery:
    def run(self):
        print('delivery')
        
w = Delivery()

with ThreadPoolExecutor() as executor:
    future = executor.submit(w.run)

delivery


In [10]:
# multiprocessing.Pool 사용
from multiprocessing import Pool
from os import getpid

def double(i):
    print(f'pid:{getpid()}')
    return i*2

with Pool() as pool:
    result = pool.map(double, [1,2,3,4,5])
    print(result)

pid:75pid:77pid:76pid:74pid:78




[2, 4, 6, 8, 10]


## concurrent.futures
#### Executor
* ThreadPoolExecutor
* ProcessPoolExecutor

###### method
1. submit
2. map
3. shutdown

#### Futures

In [11]:
import math
import concurrent

# 소수인지 아닌지 판별하는 문제
PRIMES = [
    112272535095293,
    112582705942171,
    112272535095293,
    115280095190773,
    115797848077099,
    1099726899285419]

# 소수인지 아닌지 구하는 함수
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

def main():
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):
            print('%d is prime: %s' % (number, prime))

if __name__ == '__main__':
    main()

112272535095293 is prime: True
112582705942171 is prime: True
112272535095293 is prime: True
115280095190773 is prime: True
115797848077099 is prime: True
1099726899285419 is prime: False
